# Анализ метрик AIDA для канала Youtube

Бизнес все чаще использует контент-маркетинг для своего продвижения. Становится тяжелее отслеживать результаты, тренды и удачные примеры. Какой контент хочет видеть наша аудитория? О чем и как снимать? Какие результаты у нашего канала? Как это влияет на наш бизнес? Несмотря на встроенные возможности Youtube для аналитики каналов (Youtube Studio) - возможности этой платформы зачастую ограничены или не обладают необходимой для бизнеса гибкость.

**Цель исследования** - Разработать автоматическую систему анализа YouTube-канала для бизнеса. Это система должна помогать:
владельцу бизнеса и стейкхолдерам отслеживать эффективность бизнес-единицы (YouTube-канала) и позволять интегрировать данные бизнеса для расчёта дополнительных параметров (например, добавление информации о расходах для расчёта метрики **ROI/ROMI**)

**Ход исследования**

Заказчик - *Роман Сергеев*, создатель Youtube-канала с рисованными видео на книги по бизнесу, психологии и личной эффективности [**RomanSergeevCom**](https://www.youtube.com/c/RomanSergeevCom/about), на момент исследования на канал подписаны более 485 тыс. человек. Основными источниками данных станут выгрузка общей статистики канала за октябрь 2022 с разбивкой по дням и статистика всех видео канала за этот же период. Будет проведён обзор и предобработка данных - данные будут проверены на наличие дубликатов, пропущенных или аномальных значений, а также корректность типа хранимых данных. После чего для данных будут расчитаны метрики по модели [**AIDA**](https://en.wikipedia.org/wiki/AIDA_(marketing)) (согласовано с заказчиком), но основе которых будет проведён исследовательский анализ, в ходе которого будет изучено распределение значений метрик в наборах данных. По результатам будет сделан общий вывод, после чего датасеты будут выгружены для создания интерактивного дашборда.

Таким образом, исследование будет разбито на следующие этапы:


1. [**Обзор и предобработка данных**](#1)
2. [**Исследовательский анализ данных**](#2)
3. [**Общий вывод**](#3)

Для выполнения проекта будет использованы следующие инструменты:

- **Предобработка/исследовательский анализ** - библиотеки для анализа и визуализации (pandas/numpy/matplotlib/seaborn)
- **Дашборд** - Tableau Public

<a id='1'></a>
## Обзор и предобработка данных

In [2]:
# импортируем библиотеки для работы с данными

import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
from matplotlib import pyplot as plt

# настройки отображения

pd.options.display.float_format = '{:,.2f}'.format
pd.options.mode.chained_assignment = None
sns.set_style('darkgrid')
plt.rcParams.update({'axes.labelsize': 12,'axes.titlesize': 16})
plt.rcParams['figure.figsize'] = [15, 5]

# путь к датасету

import os
os.chdir('C:\\Users\\dmitr\\GitHub\\Pet-Projects\\datasets')

In [21]:
# загружаем первый датафрейм (общая статистика канала), выводим общую информацию, описательную статистику 10 случайных строк
# первая строка в сырых данных содержит итоги столбцов, исключим её из выгрузки

overall = pd.read_excel('youtube_overall_october.xlsx').drop(index=0)

overall.info()
display(overall.describe())
overall.sample(10, random_state=0)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 1 to 31
Data columns (total 24 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Date                                     31 non-null     object 
 1   Clicks per end screen element shown (%)  31 non-null     float64
 2   End screen element clicks                31 non-null     float64
 3   Teaser clicks per card teaser shown (%)  31 non-null     float64
 4   Card teaser clicks                       31 non-null     float64
 5   Clicks per card shown (%)                28 non-null     float64
 6   Card clicks                              31 non-null     float64
 7   Comments added                           31 non-null     float64
 8   Shares                                   31 non-null     float64
 9   Dislikes                                 31 non-null     float64
 10  Likes                                    31 non-null

,Clicks per end screen element shown (%),End screen element clicks,Teaser clicks per card teaser shown (%),Card teaser clicks,Clicks per card shown (%),Card clicks,Comments added,Shares,Dislikes,Likes,...,New viewers,Unique viewers,Impressions click-through rate (%),Impressions,Videos published,Videos added,Subscribers,Average percentage viewed (%),Views,Watch time (hours)
count,31.00,31.00,31.00,31.00,28.00,31.00,31.00,31.00,31.00,31.00,...,31.00,31.00,31.00,31.00,31.00,31.00,31.00,31.00,31.00,31.00
mean,1.84,30.45,0.28,1.84,12.18,0.55,2.06,90.84,2.68,110.29,...,"1,545.42","2,493.61",4.49,"45,950.87",0.03,0.03,3.26,31.88,"3,454.03",265.07
std,0.36,7.00,0.23,1.53,14.23,0.68,2.24,18.35,2.52,37.48,...,219.19,530.57,0.36,"9,907.68",0.18,0.18,15.57,1.01,682.07,43.40
min,1.23,18.00,0.00,0.00,0.00,0.00,0.00,59.00,-1.00,57.00,...,"1,172.00","1,983.00",3.73,"38,401.00",0.00,0.00,-49.00,29.10,"2,766.00",200.90
25%,1.61,26.00,0.15,1.00,0.00,0.00,1.00,74.50,1.00,88.00,...,"1,464.50","2,283.50",4.24,"41,603.00",0.00,0.00,-4.50,31.43,"3,091.50",244.40
50%,1.77,30.00,0.25,2.00,6.67,0.00,2.00,90.00,2.00,109.00,...,"1,527.00","2,471.00",4.61,"43,525.00",0.00,0.00,3.00,31.83,"3,389.00",261.72
75%,2.15,35.50,0.36,2.00,20.00,1.00,2.50,107.00,5.00,124.50,...,"1,606.00","2,555.00",4.69,"46,484.50",0.00,0.00,14.50,32.45,"3,574.00",277.22
max,2.55,46.00,0.75,6.00,50.00,2.00,11.00,128.00,8.00,277.00,...,"2,445.00","5,129.00",5.20,"94,418.00",1.00,1.00,25.00,33.77,"6,776.00",451.42


,Date,Clicks per end screen element shown (%),End screen element clicks,Teaser clicks per card teaser shown (%),Card teaser clicks,Clicks per card shown (%),Card clicks,Comments added,Shares,Dislikes,...,Unique viewers,Impressions click-through rate (%),Impressions,Videos published,Videos added,Subscribers,Average percentage viewed (%),Views,Watch time (hours),Average view duration
3,2022-10-01,2.29,33.00,0.54,3.00,25.00,2.00,1.00,65.00,1.00,...,"1,983.00",3.73,"44,269.00",0.00,0.00,-2.00,30.91,"2,860.00",204.94,0:04:17
30,2022-10-24,1.23,20.00,0.14,1.00,0.00,0.00,4.00,109.00,-1.00,...,"2,473.00",4.77,"45,947.00",0.00,0.00,17.00,31.49,"3,574.00",268.29,0:04:30
14,2022-10-03,1.90,31.00,0.25,2.00,25.00,1.00,3.00,74.00,6.00,...,"2,513.00",4.15,"46,413.00",0.00,0.00,21.00,31.39,"3,346.00",259.78,0:04:39
11,2022-10-17,2.03,39.00,0.00,0.00,0.00,0.00,2.00,85.00,2.00,...,"2,654.00",5.20,"42,871.00",0.00,0.00,21.00,32.16,"3,765.00",301.23,0:04:48
28,2022-10-11,1.38,25.00,0.74,5.00,14.29,1.00,3.00,107.00,0.00,...,"2,502.00",4.70,"44,633.00",0.00,0.00,1.00,31.74,"3,542.00",277.78,0:04:42
26,2022-10-04,1.45,43.00,0.00,0.00,0.00,0.00,11.00,111.00,8.00,...,"5,129.00",4.14,"94,418.00",1.00,1.00,-49.00,32.43,"6,776.00",451.42,0:03:59
23,2022-10-08,1.62,20.00,0.00,0.00,NaN,0.00,5.00,95.00,2.00,...,"2,012.00",3.90,"42,394.00",0.00,0.00,-10.00,29.10,"2,766.00",200.90,0:04:21
12,2022-10-25,1.96,28.00,0.16,1.00,33.33,1.00,2.00,82.00,5.00,...,"2,334.00",4.81,"38,401.00",0.00,0.00,-12.00,31.81,"3,152.00",248.79,0:04:44
18,2022-10-26,1.75,26.00,0.27,2.00,20.00,1.00,0.00,111.00,5.00,...,"2,334.00",4.68,"40,235.00",0.00,0.00,13.00,31.13,"3,266.00",248.69,0:04:34
24,2022-10-18,1.60,29.00,0.29,2.00,16.67,1.00,1.00,123.00,3.00,...,"2,604.00",4.72,"44,006.00",0.00,0.00,12.00,33.42,"3,588.00",299.04,0:05:00


In [22]:
# загружаем второй датафрейм (статистика по каждому видео), выводим общую информацию и 10 случайных строк
# первая строка в сырых данных содержит итоги столбцов, исключим её из выгрузки

content = pd.read_excel('youtube_content_october.xlsx').drop(index=0)

content.info()
display(content.describe())
content.sample(10, random_state=0)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220 entries, 1 to 220
Data columns (total 24 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Content                                  220 non-null    object 
 1   Video title                              220 non-null    object 
 2   Video publish time                       191 non-null    object 
 3   Clicks per end screen element shown (%)  178 non-null    float64
 4   End screen element clicks                195 non-null    float64
 5   Teaser clicks per card teaser shown (%)  176 non-null    float64
 6   Card teaser clicks                       195 non-null    float64
 7   Card clicks                              195 non-null    float64
 8   Clicks per card shown (%)                39 non-null     float64
 9   Subscribers gained                       195 non-null    float64
 10  Unique viewers                           220 non-n

,Clicks per end screen element shown (%),End screen element clicks,Teaser clicks per card teaser shown (%),Card teaser clicks,Card clicks,Clicks per card shown (%),Subscribers gained,Unique viewers,Returning viewers,New viewers,Average percentage viewed (%),Subscribers,Comments added,Shares,Dislikes,Likes,Views,Watch time (hours),Impressions,Impressions click-through rate (%)
count,178.00,195.00,176.00,195.00,195.00,39.00,195.00,220.00,220.00,220.00,194.00,195.00,195.00,195.00,195.00,195.00,195.00,195.00,220.00,217.00
mean,1.66,4.84,0.32,0.29,0.09,10.23,7.51,400.65,149.30,251.35,35.86,6.93,0.33,14.44,0.43,17.53,549.10,42.14,"6,474.91",3.88
std,1.94,9.50,0.92,1.08,0.55,21.46,18.34,861.37,352.25,574.62,10.99,18.43,1.11,28.40,1.31,33.58,"1,088.48",94.20,"17,507.34",2.62
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.63,-48.00,0.00,0.00,-2.00,-1.00,0.00,0.00,0.00,0.00
25%,0.00,0.00,0.00,0.00,0.00,0.00,0.00,41.75,18.00,19.00,27.21,0.00,0.00,2.00,0.00,3.00,82.00,6.31,888.75,2.13
50%,1.17,1.00,0.00,0.00,0.00,0.00,2.00,156.50,60.50,85.50,35.61,2.00,0.00,5.00,0.00,7.00,237.00,15.06,"2,369.50",4.00
75%,2.21,5.00,0.00,0.00,0.00,5.55,6.00,352.50,149.25,220.25,43.61,6.00,0.00,15.00,0.00,16.00,554.00,38.56,"4,767.25",4.96
max,11.59,62.00,6.47,13.00,7.00,100.00,151.00,"6,807.00","4,154.00","4,983.00",67.98,145.00,12.00,183.00,14.00,291.00,"8,090.00",842.76,"167,318.00",18.70


,Content,Video title,Video publish time,Clicks per end screen element shown (%),End screen element clicks,Teaser clicks per card teaser shown (%),Card teaser clicks,Card clicks,Clicks per card shown (%),Subscribers gained,...,Subscribers,Comments added,Shares,Dislikes,Likes,Views,Watch time (hours),Average view duration,Impressions,Impressions click-through rate (%)
153,DFLRIY0ir1o,«Конор Макгрегор». Джон Кавана | Саммари ®,"Oct 4, 2018",NaN,0.00,0.00,0.00,0.00,NaN,1.00,...,1.00,0.00,0.00,0.00,3.00,74.00,3.90,0:03:09,"1,276.00",4.00
75,-szfQJ1-9OM,«Привычка достигать». Бернард Рос | Саммари,"Sep 12, 2017",1.56,2.00,0.00,0.00,0.00,NaN,4.00,...,4.00,0.00,13.00,0.00,11.00,320.00,28.35,0:05:18,"5,173.00",4.18
72,1HFRquFwhIM,«Первоклассный сервис как конкурентное преимущ...,"Jul 28, 2021",1.49,3.00,0.00,0.00,0.00,NaN,3.00,...,2.00,0.00,13.00,0.00,8.00,325.00,23.08,0:04:15,"9,172.00",2.07
162,erxfLffk8IQ,Взрывной рост компании «Кнопка»,"Dec 28, 2019",0.00,0.00,0.00,0.00,0.00,NaN,0.00,...,0.00,0.00,0.00,0.00,3.00,54.00,4.49,0:04:59,"1,731.00",1.79
163,Sd97h4xhepo,«Отдел продаж по захвату рынка». Часть 3. Миха...,"Oct 31, 2019",0.00,0.00,0.00,0.00,0.00,NaN,0.00,...,0.00,0.00,0.00,0.00,2.00,52.00,3.84,0:04:25,"1,482.00",1.96
144,z41Du_6vDHI,«Уоррен Баффет» Элис Шрёдер | Саммари,"Jan 23, 2018",0.00,0.00,0.00,0.00,0.00,NaN,0.00,...,0.00,0.00,1.00,0.00,5.00,88.00,13.72,0:09:21,"1,442.00",2.64
64,APZqHC1Jo0s,«Красная Таблетка» Андрей Курпатов | (АНИМАЦИЯ),"May 26, 2018",2.06,7.00,0.00,0.00,0.00,NaN,2.00,...,2.00,0.00,5.00,1.00,6.00,367.00,14.02,0:02:17,"2,833.00",8.82
154,fUPNozpFav8,«Золотая лихорадка». Владислав Мусатов | Самма...,"Feb 13, 2019",0.00,0.00,0.00,0.00,0.00,NaN,4.00,...,4.00,0.00,5.00,0.00,2.00,74.00,5.22,0:04:13,"1,196.00",2.34
220,ygIEgoCZFnA,Antonio 07 Acoustic | Music by RomanSergeevCom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.00,0.00
136,BJuRCUn5Xt4,«Экономика впечатлений» Джозеф Пайн и Джеймс Г...,"Oct 6, 2019",3.09,3.00,0.00,0.00,0.00,NaN,2.00,...,2.00,0.00,6.00,0.00,5.00,117.00,7.79,0:03:59,"1,669.00",3.06


- `Impressions`	How many times your video thumbnails were shown to viewers. Includes only impressions on YouTube, not on external sites or apps.
- `CTR (Impressions click-through rate)`	Views per impressions shown. This measures how often viewers watched a video after seeing an impression. Wondering whether your Impressions click-through rate is high or low?	Сведения о том, как часто пользователи просматривают видео после того, как увидят его значок.
- `Views`	Total views for the selected date range, region and other filters.	
- `Unique viewers`	Estimated number of people that watched your content within the selected date range.	
- `New viewers`	Viewers who discovered your channel for the first time by watching this video	
- `Returning viewers`		Постоянные зрители
		
- `Average view duration`	Estimated average minutes watched per view for the selected content, date range, region and other filters.	Данные о среднем времени просмотра по выбранному видео или временному диапазону.
- `Watch time (hours)`	Estimated total hours of viewing time of your content from your audience.	Общее время, в течение которого пользователи смотрели ваши видео.
		
	
- `Likes`	The change in total likes found by subtracting "likes removed" from "likes added" for the selected date range, region and other filters. Prior to September 1, 2012, "likes removed" were not subtracted.	
- `Comments added`	The number of comments added on your video or channel. This count may include deleted comments as well as live chat.	
- `Subscribers gained`	The total number of times users subscribed to this channel for the selected date range and region.
		
		
- `Card clicks`	The number of times a card has been clicked.	
- `Clicks per card shown`	The average card clicks per impression. This measures how often viewers clicked a card after seeing it.	
- `Card teaser clicks`	The number of times a teaser has been clicked. Card icon clicks get attributed to the last teaser shown.	
- `Teaser clicks per card teaser shown`	The average teaser clicks per teaser impression. It measures how often viewers clicked a teaser after seeing it.	

- `clicks per end screen element shown (%)`	How often viewers clicked an end screen element when it was shown.
- `End screen element clicks`	The number of times an end screen element was clicked.
- `teaser clicks per card teaser shown (%)`	The average teaser clicks per teaser impression. It measures how often viewers clicked a teaser after seeing it.
- `card teaser clicks`- The number of times a teaser has been clicked. Card icon clicks get attributed to the last teaser shown.
- `clicks per card shown (%)` - The average card clicks per impression. This measures how often viewers clicked a card after seeing it.
- `card clicks`	The number of times a card has been clicked